In [1]:
import wandb
import torch
# from utils2 import eval_zs_gzsl
from helper_func import eval_zs_gzsl
from dataset_test import CUBDataLoader
import argparse
import json
import pickle
import os
import pickle
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader
from PIL import Image
import sys

import torch.optim as optim
import torch.nn as nn
import numpy as np
from model_test import BDA
# from dataset import AWA2DataLoader
from  helper_func import eval_zs_gzsl
import json
import argparse

In [2]:
dataloader = CUBDataLoader('.', "cuda:0")

Balance dataloader
_____
./../revice_Transzero/data/CUB/feature_map_ResNet_101_CUB.hdf5
Expert Attr


In [6]:
sweep_configuration = {
    'method': 'grid',
    'name': 'test_cub_gzsl',
    'metric': {'goal': 'maximize', 'name': 'best_H'},
    'parameters': 
    {   'batch_size':    {'value': 50},
        'dataset': { 'value': 'CUB'},
        'device': {'value': 'cuda:0'},
        'dim_f': { 'value': 2048},
        'dim_v':{ 'value': 300},
        'epochs': {'value': 1},
        'img_size': {'value': 448},
        'lambda_': {'value':  0.3},
        'lambda_reg': {'value': 0.005},
        'lambda_CEVi': {'value': 0.1},
        'normalize_V': {'value': "False"},
        'num_attribute': { 'value': 312},
        'num_class': {'value': 200},
        'random_seed': {'value': 5},
        'tf_SAtt':  {'value': "True"},
        'tf_aux_embed': {'value': "True"},
        'tf_common_dim': {'value': 377},
        'tf_dc_layer': {'value': 1},
        'tf_dim_feedforward': {'value': 377},
        'tf_dropout': {'values': [0.4]},
        'tf_dropout1': {'values': [1]},
        
        
        'tf_ec_layer': {'value': 1},
        'tf_heads': {'value': 1},
        'alpha': {'value': 0},
        'beta': {'value': 1},
        'fsqrt': {'values': [155]},
        'sqrt': {'values': [155]},
        'weight' : {'values': [1]},
        'weight_self' : {'values': [1]},
        'tgt_weight' : {'values': [1]}           
        }}                              
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Ko_bda')

Create sweep with ID: 5kgho6j9
Sweep URL: https://wandb.ai/junseoklee_333/Ko_bda/sweeps/5kgho6j9


In [7]:
def main():
    run = wandb.init()
    config = wandb.config
    # set random seed
    seed = config.random_seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    # TransZero model
    model = BDA(config, dataloader.att, dataloader.w2v_att,
                    dataloader.seenclasses, dataloader.unseenclasses).to(config.device)
       
    # main loop
    niters = dataloader.ntrain * config.epochs//config.batch_size
    report_interval = niters//config.epochs
    best_performance = [0, 0, 0, 0]
    best_performance_zsl = 0
    model_dict = model.state_dict()
    # saved_dict = torch.load(config.saved_model)
    check_point = torch.load("./checkpoint/CUB/CUB_gzsl.pth")
    saved_dict = check_point["model_state_dict"]
    saved_dict = {k: v for k, v in saved_dict.items() if k in model_dict}
    model_dict.update(saved_dict)
    model.load_state_dict(model_dict)
    model.to(config.device)
    model.eval()

    # report result

    print('-'*30)
    acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(
        dataloader, model, config.device, bias_seen=0, bias_unseen=0)

    if H > best_performance[2]:
        best_performance = [acc_novel, acc_seen, H, acc_zs]
    if acc_zs > best_performance_zsl:
        best_performance_zsl = acc_zs

    print('acc_unseen=%.3f, acc_seen=%.3f, H=%.3f | '
        'acc_zs=%.3f' % (  best_performance[0], best_performance[1],
            best_performance[2], best_performance_zsl))

    wandb.log({   
        'best_acc_unseen': best_performance[0],
        'best_acc_seen': best_performance[1],
        'best_H': best_performance[2],
        'best_acc_zs': best_performance_zsl
        })

In [8]:
wandb.agent(sweep_id, function=main, count=2 )

wandb: Agent Starting Run: h26z6df6 with config:
wandb: 	alpha: 0
wandb: 	batch_size: 50
wandb: 	beta: 1
wandb: 	dataset: CUB
wandb: 	device: cuda:0
wandb: 	dim_f: 2048
wandb: 	dim_v: 300
wandb: 	dropout_self: 1
wandb: 	epochs: 1
wandb: 	fsqrt: 155
wandb: 	img_size: 448
wandb: 	lambda_: 0.3
wandb: 	lambda_CEVi: 0.1
wandb: 	lambda_reg: 0.005
wandb: 	normalize_V: False
wandb: 	num_attribute: 312
wandb: 	num_class: 200
wandb: 	random_seed: 5
wandb: 	sqrt: 155
wandb: 	tf_SAtt: True
wandb: 	tf_aux_embed: True
wandb: 	tf_common_dim: 377
wandb: 	tf_dc_layer: 1
wandb: 	tf_dim_feedforward: 377
wandb: 	tf_dropout: 0.4
wandb: 	tf_dropout1: 1
wandb: 	tf_ec_layer: 1
wandb: 	tf_heads: 1
wandb: 	tgt_weight: 1
wandb: 	weight: 1
wandb: 	weight_self: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


------------------------------
acc_unseen=0.695, acc_seen=0.698, H=0.697 | acc_zs=0.763


H,▁
acc_seen,▁
acc_unseen,▁
acc_zs,▁
best_H,▁
best_acc_seen,▁
best_acc_unseen,▁
best_acc_zs,▁
H,0.69667
acc_seen,0.69814
acc_unseen,0.69521


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
    # load parameters
model_dict = model.state_dict()
# saved_dict = torch.load(config.saved_model)
check_point = torch.load("./checkpoint/AWA2/AWA2_gzsl.pth")
saved_dict = check_point["model_state_dict"]
saved_dict = {k: v for k, v in saved_dict.items() if k in model_dict}
model_dict.update(saved_dict)
model.load_state_dict(model_dict)
model.to(config.device)

In [ ]:
acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(
    dataloader, model, config.device, bias_seen=0, bias_unseen=0)
best_performance = [acc_novel, acc_seen, H, acc_zs]

In [ ]:
print(' acc_unseen=%.3f, acc_seen=%.3f, H=%.3f' %( best_performance[0], best_performance[1],
        best_performance[2]))

In [3]:
def run_test(config):
    print(f'Dataset: {config.dataset}\nSetting: {config.zsl_task}')
    # dataset
    
    # model
    model = TransZero(config)
    # load parameters
    model_dict = model.state_dict()
    # saved_dict = torch.load(config.saved_model)
    check_point = torch.load("./checkpoint/SUN/SUN_GZSL.pth")
    saved_dict = check_point["model_state_dict"]
    saved_dict = {k: v for k, v in saved_dict.items() if k in model_dict}
    model_dict.update(saved_dict)
    model.load_state_dict(model_dict)
    model.to(config.device)
    # evaluation
    evaluation(config, dataloader, model)

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./wandb_config/sun_gzsl.json')
# config = parser.parse_known_args()
config, _ = parser.parse_known_args()
print(config.config)
with open(config.config, 'r') as f:
    config.__dict__ = json.load(f)
run_test(config)

./wandb_config/sun_gzsl.json
Dataset: SUN
Setting: GZSL
Results: Acc_Unseen=0.000, Acc_Seen=0.012, H=0.000
